In [1]:
/* 24-8 */
USE AdventureWorks2014;
GO
UPDATE STATISTICS Sales.Customer 
WITH FULLSCAN;

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.145

In [2]:
/* partitioned table setup from chapter 15 */

EXECUTE xp_create_subdir 'C:\Apress'; 

IF DB_ID('MegaCorpData') IS NOT NULL DROP DATABASE MegaCorpData; 
GO

Commands completed successfully.

Total execution time: 00:00:00.007

In [3]:
CREATE DATABASE MegaCorpData 
ON PRIMARY 
(NAME = 'MegaCorpData', 
 FILENAME = 'C:\Apress\MegaCorpData.MDF', 
 SIZE = 3MB, 
 MAXSIZE = UNLIMITED, 
 FILEGROWTH = 1MB) 
LOG ON 
(NAME = 'MegaCorpData_Log', 
 FILENAME = 'C:\Apress\MegaCorpData.LDF', 
 SIZE = 3MB, 
 MAXSIZE = UNLIMITED, 
 FILEGROWTH = 1MB); 
GO 
ALTER DATABASE MegaCorpData ADD FILEGROUP hitfg1; 
ALTER DATABASE MegaCorpData ADD FILEGROUP hitfg2; 
ALTER DATABASE MegaCorpData ADD FILEGROUP hitfg3; 
ALTER DATABASE MegaCorpData ADD FILEGROUP hitfg4; 
ALTER DATABASE MegaCorpData  
ADD FILE (NAME = mchitfg1,  
          FILENAME = 'C:\Apress\mc_hitfg1.ndf',  
          SIZE = 1MB)  
TO FILEGROUP hitfg1; 
ALTER DATABASE MegaCorpData  
ADD FILE (NAME = mchitfg2,  
          FILENAME = 'C:\Apress\mc_hitfg2.ndf',  
          SIZE = 1MB)  
TO FILEGROUP hitfg2; 
ALTER DATABASE MegaCorpData  
ADD FILE (NAME = mchitfg3,  
          FILENAME = 'C:\Apress\mc_hitfg3.ndf',  
          SIZE = 1MB)  
TO FILEGROUP hitfg3; 
ALTER DATABASE MegaCorpData  
ADD FILE (NAME = mchitfg4,  
          FILENAME = 'C:\Apress\mc_hitfg4.ndf',  
          SIZE = 1MB)  
TO FILEGROUP hitfg4; 
USE MegaCorpData; 
GO 
CREATE PARTITION FUNCTION HitsDateRange (datetime) 
AS RANGE LEFT FOR VALUES ('2006-01-01T00:00:00', '2007-01-01T00:00:00', '2008-01-01T00:00:00'); 
CREATE PARTITION SCHEME HitDateRangeScheme 
AS PARTITION HitsDateRange 
TO (hitfg1, hitfg2, hitfg3, hitfg4); 

CREATE TABLE dbo.WebSiteHits ( 
    WebSiteHitID BIGINT NOT NULL IDENTITY(1, 1),  
    WebSitePage VARCHAR(255) NOT NULL, 
    HitDate DATETIME NOT NULL, 
    CONSTRAINT PK_WebSiteHits PRIMARY KEY CLUSTERED (WebSiteHitId, HitDate) 
) 
ON [HitDateRangeScheme] (HitDate); 

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.621

In [4]:
/* update by partition */
USE MegaCorpData;
GO
UPDATE STATISTICS dbo.WebSiteHits
WITH INCREMENTAL = ON;

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.020